<a href="https://colab.research.google.com/github/BrianKEverett/County-Line/blob/main/Dissertation6_Everett.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ydata-profiling
from ydata_profiling import ProfileReport

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.1 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=e8b05e4ee09e4f1df763eb03b9ffaa852aa4a0aa5e7c577ab1c6b65e30f5603b
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [2]:
import time, os, sys, re
import zipfile, json, datetime, string
import numpy as np
from statistics import *

import matplotlib.pyplot as plt

from scipy.stats import chi2_contingency
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
import plotly.express as px

import missingno as msno

from google.colab import files

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table.max_columns = 50

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

plt.style.use('classic')

In [3]:
permits=pd.read_csv("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/3.6.2024.NJ_Construction_Permit_Data.csv")
# dataset can be found here: https://data.nj.gov/Reference-Data/NJ-Construction-Permit-Data/w9se-dmra/explore/query/SELECT%0A%20%20%60comu%60%2C%0A%20%20%60muniname%60%2C%0A%20%20%60county%60%2C%0A%20%20%60permitstatusdesc%60%2C%0A%20%20%60permitdate%60%2C%0A%20%20%60certdate%60%2C%0A%20%20%60permittypedesc%60%2C%0A%20%20%60certtypedesc%60%2C%0A%20%20%60constcost%60%2C%0A%20%20%60usegroupdesc%60%0AWHERE%0A%20%20%28%60permitdate%60%0A%20%20%20%20%20BETWEEN%20%222015-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20AND%20%222024-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%29%0A%20%20AND%20%28%60constcost%60%20%3E%20%2249999%22%29/page/filter
#from NJ Dept of Community Affairs Website.Pulled from web on March 6th, 2024 with queries for construction costs higher than $49,999 and for dates ranging between January1, 2015 and January 1, 2024

taxes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/mediantax.csv")
# dataset can be retireved via: https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b from NJ Department of Community Affairs
# Another hypothesis I have is that the public narrative put forth by planning board members when approving controversial permits, "this will increase rateables for the township, lowering your taxes", does not actually come to fruition
#This data set is helpful for exploring that narrative, and seeing if the opposite is occurring, i.e. more development actually yields higher property taxes

municodes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Municodes.csv")
#Data file of all NJ municiaplities, and counties, with the corresponding municipalitiy DCA code. This data file will be most helpful for matching and merging.

#crime=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/CamdenCrime.csv") #not a good format for reading data!
#Dataset can be retireved here: https://www.nj.gov/njsp/ucr/uniform-crime-reports.shtml on the NJ Office of the Attorney General's website
#This crime data is important to consider when analyzing planning and zoning. Does any specific type of development correlate with increased crime? Can liveability theory be worked in here for whether or not communities have what they need to prevent crime?

jobs=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/jobsdensity.csv')
#dataset can be built via the table selections on the NJ Community Affairs website - https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b
#Job density is a good variable to consider regarding new large dollar permits. Are some places growing more than others? Can this be attribute to the phenomenon of the County Line?

countysize=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/NJCountySize.csv')
#dataset can be found via Wikipedia via 2020 census data - https://en.wikipedia.org/wiki/List_of_counties_in_New_Jersey

njtowns=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/njtowns.csv')
#Dataset can be found at: https://en.wikipedia.org/wiki/List_of_municipalities_in_New_Jersey#:~:text=The%20largest%20municipality%20by%20population,most%20populous%20being%20South%20Carolina.

njcodes=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Municipalities_of_New_Jersey_20240131.csv')
#data was retireved via State of New Jersey Open Data Center: https://data.nj.gov/Reference-Data/Municipalities-of-New-Jersey/k9xb-zgh4/data_preview

In [4]:
permits

,Muni Code,Municipality Name,County,Permit Status Description,Permit Date,Certificate Date,Permit Type Description,Certificate Type Description,Construction Cost,Use Group Description
0,101,ABSECON,ATLANTIC,Certificate,03/01/2019,04/03/2019,Alteration,Certificate of Approval,50000,International Residential Code
1,101,ABSECON,ATLANTIC,Permit,04/08/2021,NaN,New,NaN,50000,Mercantile buildings
2,101,ABSECON,ATLANTIC,Permit,01/03/2023,NaN,Addition,NaN,50000,Accessory buildings and miscellaneous structures
3,102,ATLANTIC CITY,ATLANTIC,Permit,01/22/2019,NaN,Alteration,NaN,50000,Business Uses
4,102,ATLANTIC CITY,ATLANTIC,Permit,05/02/2019,NaN,Alteration,NaN,50000,Accessory buildings and miscellaneous structures
...,...,...,...,...,...,...,...,...,...,...
204939,906,JERSEY CITY,HUDSON,Certificate,10/15/2019,03/30/2022,New,Certificate of Occupancy,279500000,"Multiple family dwellings, dormitories, etc."
204940,906,JERSEY CITY,HUDSON,Certificate,03/04/2020,11/23/2021,New,Certificate of Occupancy,400949780,"Multiple family dwellings, dormitories, etc."
204941,211,ELMWOOD PARK,BERGEN,Permit,03/01/2023,NaN,New,NaN,412460000,Mercantile buildings
204942,906,JERSEY CITY,HUDSON,Permit,01/12/2023,NaN,New,NaN,621575900,International Residential Code


In [5]:
#Does the above warning mean it is just limiting what is shown, or, does it mean the excess rows are not even callable for the data set, aka data loss?